In [1]:
import pandas as pd
import numpy as np
import re
import networkx
from networkx.algorithms.components.connected import connected_components
from collections import defaultdict
import random
import glob
import os
from Bio import SeqIO
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import jaccard_score
import time
start = time.time()

# Loading MIBiG dataframes and creating dictionaries

In [2]:
bioactivity_df = pd.read_csv("./bioactivity_df-220203.csv",sep='\t',index_col=0)

bioactivity_dict = dict(zip(bioactivity_df['metabolite'],bioactivity_df['activity']))

bioactivity_dict['abyssomicin']

'antibacterial'

In [3]:
np.unique(list(bioactivity_dict.values()))

array(['antibacterial', 'antibacterial-antifungal',
       'antibacterial-antifungal-cytotoxic', 'antibacterial-cytotoxic',
       'antifungal', 'antifungal-cytotoxic', 'cytotoxic',
       'cytotoxic-unknown', 'nan', 'unknown'], dtype='<U34')

In [4]:
mibig_df = pd.read_csv("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/mibig_classifications/All_MIBiG_compounds_with_CF_NPC_classes.txt",sep='\t')

mibig_df

,compound_name,smiles,inchi_key,cf_kingdom,cf_superclass,cf_class,cf_subclass,cf_direct_parent,npc_class,npc_superclass,npc_pathway,npc_isglycoside
0,BGC0000001_abyssomicin C,CC1C[C@]23OC(=O)C4=C2OC1C(O)C3\C=C/C(=O)[C@@H]...,FNEADFUPWHAVTA-UHFFFAOYSA-N,Organic compounds,Organoheterocyclic compounds,Oxanes,NaN,Oxanes,Spirotetronate macrolides,Macrolides,Polyketides,0
1,BGC0000001_atrop-abyssomicin C,CC1CC23OC(=O)C4=C2OC1C(O)C3\C=C/C(=O)C(C)CC(C)...,FNEADFUPWHAVTA-UHFFFAOYSA-N,Organic compounds,Organoheterocyclic compounds,Oxanes,NaN,Oxanes,Spirotetronate macrolides,Macrolides,Polyketides,0
2,BGC0000002_aculeximycin,CCCC(O[C@H]1C[C@](C)(N)[C@H](O)[C@H](C)O1)C(C)...,VJKZKLDZOAFAEE-UHFFFAOYSA-N,Organic compounds,Lipids and lipid-like molecules,Prenol lipids,Terpene glycosides,Diterpene glycosides,NaN,NaN,NaN,1
3,BGC0000003_AF-toxin,CCC(C)C(C(=O)OC(/C=C/C=C/C=C/C(=O)O)C1(CO1)C)O...,ONOBRFRRMLDPES-UHFFFAOYSA-N,Organic compounds,Organic acids and derivatives,Peptidomimetics,Depsipeptides,Depsipeptides,NaN,NaN,NaN,0
4,BGC0000004_aflatoxin G1,[H][C@@]12OC=C[C@]1([H])C1=C(O2)C=C(OC)C2=C1OC...,XWIYFDMXXLINPU-UHFFFAOYSA-N,Organic compounds,Phenylpropanoids and polyketides,Coumarins and derivatives,Furanocoumarins,Difurocoumarolactones,Aflatoxins; Simple coumarins,Chromanes; Coumarins,Polyketides; Shikimates and Phenylpropanoids,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2107,BGC0002034_perquinoline A,C1CC2(C(C3=C(C=C(C=C3O)O)C(N2C1=O)C(=O)NCCCC(=...,CDFQOHHGTNUFPU-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alkaloids,0
2108,BGC0002034_perquinoline B,C1CC2(C(C3=C(C=C(C=C3O)O)C(N2C1=O)C(=O)NCCC(=O...,XHJZZGRZEGUFCM-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alkaloids,0
2109,BGC0002034_perquinoline C,C1CC2(C(C3=C(C=C(C=C3O)O)C(N2C1=O)C(=O)NCCCC(=...,ZNSLKIGPUIPFNC-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alkaloids,0
2110,BGC0002035_ilicicolin H,CC=CC1C2CC(CCC2C(=CC1C(=O)C3=C(C(=CNC3=O)C4=CC...,BYVVOONSAAQMKI-UHFFFAOYSA-N,Organic compounds,Organoheterocyclic compounds,Pyridines and derivatives,Phenylpyridines,Phenylpyridines,Pyridine alkaloids,Nicotinic acid alkaloids,Alkaloids,0


In [5]:
mibig_activity_dict = {}
missing_mets_list = []

for i,r in mibig_df.iterrows():
    met_name = r['compound_name'].split('_')[1].replace(' ','_').lower().split('_')[0]
    for key in bioactivity_dict.keys():
        if type(key) != float:
            if met_name in key:
                if type(bioactivity_dict[key]) == float:
                    mibig_activity_dict[r['compound_name'].split('_')[0]] = 'unknown'
                else:
                    mibig_activity_dict[r['compound_name'].split('_')[0]] = bioactivity_dict[key]
            else:
                missing_mets_list.append(met_name)
                
mibig_activity_dict['BGC0000001']

'antibacterial'

In [6]:
len(mibig_activity_dict)

806

In [7]:
mibig_bio_3_df = pd.read_csv('./mibig_3_bioactivity.csv',sep='\t',names=['MIBIG_ID','Activity'])

mibig_bio_3_dict = dict(zip(mibig_bio_3_df['MIBIG_ID'],mibig_bio_3_df['Activity']))

mibig_bio_3_dict['BGC0000018']

'Antibacterial'

In [8]:
for key in mibig_activity_dict:
    if mibig_activity_dict[key] == 'unknown':
        if key in mibig_bio_3_dict.keys():
            mibig_activity_dict[key] = mibig_bio_3_dict[key].lower()

for key in mibig_bio_3_dict:
    if key not in mibig_activity_dict:
        mibig_activity_dict[key] = mibig_bio_3_dict[key].lower()

In [9]:
bgc_subtype_df = pd.read_csv("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/NRPOmix/bgc_subtype_df.csv",'\t',names=['BGC','subtype'])

bgc_subtype_df[:5]

,BGC,subtype
0,BGC0000001,Modular type I polyketide
1,BGC0000002,Polyketide
2,BGC0000003,Polyketide
3,BGC0000004,Polyketide
4,BGC0000005,Polyketide


In [10]:
subtype_type_dict = {}
with open("/Users/tiagoferreiraleao/Dropbox/tiago-NAS/NRPOmix/subtype_type_df.csv") as f:
    for line in f:
        (key, val) = line.split(',')
        subtype_type_dict[key] = val.strip('\n')
        
subtype_type_dict['Modular type I polyketide']

'PKS'

In [11]:
type_col = []

for i,r in bgc_subtype_df.iterrows():
    type_col.append(subtype_type_dict[r['subtype']])
    
bgc_subtype_df['type'] = type_col

bgc_subtype_df

,BGC,subtype,type
0,BGC0000001,Modular type I polyketide,PKS
1,BGC0000002,Polyketide,PKS
2,BGC0000003,Polyketide,PKS
3,BGC0000004,Polyketide,PKS
4,BGC0000005,Polyketide,PKS
...,...,...,...
1921,BGC0002045,Type II polyketide,PKS
1922,BGC0002055,Trans-AT type I polyketide,PKS
1923,BGC0002056,Trans-AT type I polyketide,PKS
1924,BGC0002057,Trans-AT type I polyketide,PKS


In [12]:
bgc_type_dict = dict(zip(bgc_subtype_df.BGC,bgc_subtype_df.type))

bgc_type_dict['BGC0000001']

'PKS'

# Getting classes for CENA samples

In [13]:
def get_feature_class(bgc_name,genome_name):
    filename = "./antismash_results_CENAs/%s/%s"%(genome_name,bgc_name)
    if os.path.exists(filename):
        input_handle = open(filename,'r')
        for seq_record in SeqIO.parse(input_handle,'genbank'):
            edge_list,type_list = [],[]
            for feature in seq_record.features:
                if feature.type == 'cand_cluster':
                    for qual in feature.qualifiers:
                        if qual == 'contig_edge':
                            ctg_edge = feature.qualifiers[qual]
                            edge_list.append(ctg_edge[0])
                        if qual == 'product':
                            bgc_type = feature.qualifiers[qual]
                            for item_type in bgc_type:
                                if item_type not in type_list:
                                    type_list.append(item_type)
            return edge_list,type_list
    else:
        return 'NA','NA'
            
all_classes = []

for root, dirs, files in os.walk("./antismash_results_CENAs/"):
    for file in files:
        if file.endswith(".gbk"):
            ctg_edge,class_list = get_feature_class(file,os.path.basename(root))
            for class_type in class_list:
                all_classes.append(class_type)
    
all_classes = np.unique(all_classes)

all_classes

array(['CDPS', 'LAP', 'NRPS', 'NRPS-like', 'RRE-containing', 'RiPP-like',
       'T1PKS', 'arylpolyene', 'betalactone', 'cyanobactin', 'hglE-KS',
       'ladderane', 'lanthipeptide-class-ii', 'lanthipeptide-class-v',
       'lassopeptide', 'microviridin', 'other', 'proteusin', 'resorcinol',
       'siderophore', 'spliceotide', 'terpene', 'thioamitides',
       'thiopeptide'], dtype='<U22')

In [14]:
common_dict = {"A":"NA",
"CDPS":"NRPS",
"LAP":"RiPP",
"N":"NA",
"NAGGN":"NRPS",
"NRPS":"NRPS",
"NRPS-like":"NRPS",
"PBDE":"Phenolic",
"PKS-like":"PKS",
"PUFA":"PKS",
"PpyS-KS":"PKS",
"T1PKS":"PKS",
"T2PKS":"PKS",
"T3PKS":"PKS",
"TfuA-related":"RiPP",
"amglyccycl":"Minor",
"arylpolyene":["PKS","Phenolic"],
"bacteriocin":"RiPP",
"betalactone":"Minor",
"blactam":"Minor",
"butyrolactone":"Minor",
"cyanobactin":"RiPP",
"ectoine":"Minor",
"furan":"Minor",
"fused":"Minor",
"hglE-KS":"PKS",
"hserlactone":["PKS","NRPS"],
"indole":"Minor",
"ladderane":"RiPP",
"lanthipeptide":"RiPP",
"lassopeptide":"RiPP",
"linaridin":"RiPP",
"melanin":"NRPS",
"microviridin":"RiPP",
"nucleoside":"Nucleoside",
"oligosaccharide":"Oligosaccharide",
"other":"Other",
"phenazine":"Minor",
"phosphonate":"Phosphonate",
"proteusin":"RiPP",
"resorcinol":"PKS",
"sactipeptide":"RiPP",
"siderophore":"Siderophore",
"terpene":"Terpene",
"thiopeptide":"RiPP",
"transAT-PKS":"PKS",
"transAT-PKS-like":"PKS",
"RRE-containing":"RiPP",
"RiPP-like":"RiPP",
"lanthipeptide-class-ii":"RiPP",
"lanthipeptide-class-v":"RiPP",
"spliceotide":"Minor",
"thioamitides":"RiPP"}

In [15]:
for item in all_classes:
    if item not in common_dict.keys():
        print(item)

# Creating class dataframe

In [16]:
combined_list1,all_classes,type_list = [],[],[]

for key in bgc_type_dict:
    if 'tRNA' in bgc_type_dict[key]:
        type_list = ['tRNA_derived']
    else:
        type_list = bgc_type_dict[key].split('-')
    for item in type_list:
        if item not in all_classes:
            all_classes.append(item)
    combined_list1.append(type_list)
    
combined_list1[0]

['PKS']

In [17]:
combined_list2 = []
cena_bgc_list = []

for root, dirs, files in os.walk("./antismash_results_CENAs/"):
    for file in files:
        if 'region' in file:
            if file.endswith(".gbk"):
                ctg_edge,class_list = get_feature_class(file,os.path.basename(root))
                combined_classes = []
                for bgc_class in class_list:
                    if type(common_dict[bgc_class]) != list:
                        combined_classes.append(common_dict[bgc_class])
                        if common_dict[bgc_class] not in all_classes:
                            all_classes.append(common_dict[bgc_class])
                    else:
                        for item in common_dict[bgc_class]:
                            combined_classes.append(item)
                            if item not in all_classes:
                                all_classes.append(item)
                cena_bgc_list.append(file.split('.gbk')[0])
                combined_list2.append(combined_classes)
    
combined_list2[0]

['PKS', 'PKS']

In [18]:
combined_list = combined_list1 + combined_list2

combined_list[0]

['PKS']

In [19]:
class_df = pd.DataFrame(columns=(all_classes))

bgc_list = list(bgc_type_dict.keys()) + cena_bgc_list

for i,combined_classes in enumerate(combined_list):
    row_extension = []
    for final_class in all_classes:
        if final_class in combined_classes:
            row_extension.append(1)
        else:
            row_extension.append(0)
    class_df.loc[bgc_list[i]] = row_extension
    
class_df = class_df.sort_index(axis=0)
    
class_df

,PKS,Other,Alkaloid,Oligosaccharide,Terpene,NRPS,Cyclitol,Aminocoumarin,Betalactam,Siderophore,Pyrrolobenzodiazepine,RiPP,Butyrolactone,Nucleoside,Phenazine,Aminoglycoside,tRNA_derived,Phosphonate,Minor,Phenolic
BGC0000001,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BGC0000002,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BGC0000003,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BGC0000004,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BGC0000005,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
main-chr.region008,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
main-chr.region009,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
main-chr.region010,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
main-chr.region011,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Creating similarity dataframe

In [20]:
### obtaining bigscape dataframe and bigscape dictionary

def parse_bigscape_df(input_file,no_orphans):
    bigscape_df = pd.read_csv(input_file,sep='\t')
    bigscape_df.rename(columns=lambda x: re.sub(" ","_",x), inplace=True)
    if no_orphans == 1:
        bigscape_df = bigscape_df[bigscape_df.Clustername_1 != bigscape_df.Clustername_2]
    bigscape_df.reset_index(inplace=True,drop=True)
    return bigscape_df

def get_neighbors(target,dataframe,column1,column2):
    subset1 = dataframe[(dataframe[column1]==target)]
    subcat = subset1.append(dataframe[(dataframe[column2]==target)])
    temp_list = []
    for index,row in subcat.iterrows():
        temp_list.append(subcat[column1][index])
        temp_list.append(subcat[column2][index])
    temp_list = list(np.unique(temp_list))
    return temp_list

def to_edges(l):
    it = iter(l)
    last = next(it)
    for current in it:
        yield last, current
        last = current

def to_graph(l):
    G = networkx.Graph()
    for part in l:
        G.add_nodes_from(part)
        G.add_edges_from(to_edges(part))
    return G

def get_family_dict(components_list,dataframe,dictionary,column1,column2,column3):
    count = 0
    for family in list(components_list):
        count += 1
        for fam_member in family:
            dictionary['GCF%s'%count].append(fam_member)
    return dictionary

def main_get_families(input_file):
    bigscape_df = parse_bigscape_df(input_file,1)
    strain_list = list(np.unique([bigscape_df['Clustername_1']]+[bigscape_df['Clustername_2']]))
    targets_list = np.unique([bigscape_df.Clustername_1,bigscape_df.Clustername_2])
    neighbors_list = []
    for target in targets_list:
        neighbors_list.append(get_neighbors(target,bigscape_df,'Clustername_1','Clustername_2'))
    G = to_graph(neighbors_list)
    C = connected_components(G)
    gcf_dict = defaultdict(list)
    gcf_dict = get_family_dict(C,bigscape_df,gcf_dict,'Clustername_1','Clustername_2','Raw_distance')
    return bigscape_df,gcf_dict,strain_list


bigscape_df,bigscape_dict,strain_list = main_get_families("./bigscape_all_c030_220521_1898samples.txt")
bigscape_df["Raw_distance"] = 1-bigscape_df["Raw_distance"]

bigscape_dict['GCF1']

['BGC0000009.1.region001',
 'BGC0000006.1.region001',
 'BGC0000007.1.region001',
 'BGC0000008.1.region001',
 'BGC0000004.1.region001']

In [21]:
len(bigscape_dict)

195

In [22]:
bigscape_df2 = parse_bigscape_df("./bigscape_all_c030_220521_1898samples.txt",0)
all_clusters = list(np.unique([bigscape_df2['Clustername_1']]+[bigscape_df2['Clustername_2']]))

len(all_clusters)

1753

In [23]:
strain_list_renamed = []

for item in strain_list:
    if 'BGC' in item:
        strain_list_renamed.append(item.split('.')[0])
    else:
        strain_list_renamed.append(item)
    
strain_list_renamed = list(np.unique(strain_list_renamed))

similarity_df = pd.DataFrame(columns=strain_list_renamed, index=range(0,len(strain_list_renamed)-1))
index_row = 0
row_names = []
for gcf in bigscape_dict:
    for cluster in bigscape_dict[gcf]:
        if 'BGC' in cluster:
            row_names.append(cluster.split('.')[0])
        else:
            row_names.append(cluster)
        temp_dict = {}
        if 'BGC' in cluster:
            self = cluster.split(".")[0]
        else:
            self = cluster
        temp_dict[self] = [1]
        temp_df = bigscape_df[bigscape_df.Clustername_1.str.contains(cluster) | 
                              bigscape_df.Clustername_2.str.contains(cluster)]
        for i,r in temp_df.iterrows():
            if temp_df.Clustername_1.loc[i] == cluster:
                target = temp_df.Clustername_2.loc[i]
                if 'BGC' in target:
                    target = str(target).split(".")[0]
                if target not in temp_dict:
                    temp_dict[target] = [temp_df.Raw_distance.loc[i]]
                else:
                    temp_dict[target] = temp_dict[target]+[temp_df.Raw_distance.loc[i]]
            else:
                target = temp_df.Clustername_1.loc[i]
                if 'BGC' in target:
                    target = str(target).split(".")[0]
                if target not in temp_dict.keys():
                    temp_dict[target] = [temp_df.Raw_distance.loc[i]]
                else:
                    temp_dict[target] = temp_dict[target]+[temp_df.Raw_distance.loc[i]]
        for key in temp_dict:
            if len(temp_dict[key]) > 1:
                new_value = max(temp_dict[key])
                temp_dict[key] = new_value
            else:
                temp_dict[key] = temp_dict[key][0]
        similarity_df.loc[index_row] = pd.Series(temp_dict)
        index_row += 1
similarity_df.fillna(0,inplace=True)
similarity_df.index = row_names
similarity_df = similarity_df[~similarity_df.index.duplicated(keep='first')]
similarity_df = similarity_df.sort_index(axis=0)

len(strain_list_renamed),len(similarity_df)

(519, 519)

In [24]:
similarity_df

,BGC0000004,BGC0000006,BGC0000007,BGC0000008,BGC0000009,BGC0000020,BGC0000029,BGC0000031,BGC0000034,BGC0000035,...,NZ_MTPU01000053.region001,NZ_MTPU01000054.region001,c00017_NODE_19...region001,c00023_NZ_JAEC...region001,c00035_NODE_35...region001,c00071_NODE_64...region001,chrpls1.region003,chrpls1.region005,chrpls3.region001,main-chr.region003
BGC0000004,1.000000,0.976886,0.952463,0.975877,0.880091,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
BGC0000006,0.976886,1.000000,0.972055,0.997262,0.894729,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
BGC0000007,0.952463,0.972055,1.000000,0.971047,0.893030,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
BGC0000008,0.975877,0.997262,0.971047,1.000000,0.894099,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
BGC0000009,0.880091,0.894729,0.893030,0.894099,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c00071_NODE_64...region001,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.0,0.706209
chrpls1.region003,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000
chrpls1.region005,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.745586,0.0,0.0,0.000000,0.0,1.0,0.0,0.000000
chrpls3.region001,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1.0,0.000000


In [25]:
cluster_list_renamed = []

for item in all_clusters:
    if 'BGC' in item:
        cluster_list_renamed.append(item.split('.')[0])
    else:
        cluster_list_renamed.append(item)
    
cluster_list_renamed = list(np.unique(cluster_list_renamed))

In [26]:
orphan_list = []

for cluster in cluster_list_renamed:
    if cluster not in similarity_df.index:
        orphan_list.append(cluster)
        new_col = []
        for index in similarity_df.index:
            new_col.append(0)
        similarity_df[cluster] = new_col
        
similarity_df = similarity_df.sort_index(axis=1)
        
similarity_df

,BGC0000001,BGC0000002,BGC0000003,BGC0000004,BGC0000006,BGC0000007,BGC0000008,BGC0000009,BGC0000011,BGC0000012,...,main-chr.region003,main-chr.region004,main-chr.region005,main-chr.region006,main-chr.region007,main-chr.region008,main-chr.region009,main-chr.region010,main-chr.region011,pls1.region001
BGC0000004,0,0,0,1.000000,0.976886,0.952463,0.975877,0.880091,0,0,...,0.000000,0,0,0,0,0,0,0,0,0
BGC0000006,0,0,0,0.976886,1.000000,0.972055,0.997262,0.894729,0,0,...,0.000000,0,0,0,0,0,0,0,0,0
BGC0000007,0,0,0,0.952463,0.972055,1.000000,0.971047,0.893030,0,0,...,0.000000,0,0,0,0,0,0,0,0,0
BGC0000008,0,0,0,0.975877,0.997262,0.971047,1.000000,0.894099,0,0,...,0.000000,0,0,0,0,0,0,0,0,0
BGC0000009,0,0,0,0.880091,0.894729,0.893030,0.894099,1.000000,0,0,...,0.000000,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c00071_NODE_64...region001,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0.706209,0,0,0,0,0,0,0,0,0
chrpls1.region003,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0.000000,0,0,0,0,0,0,0,0,0
chrpls1.region005,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0.000000,0,0,0,0,0,0,0,0,0
chrpls3.region001,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0.000000,0,0,0,0,0,0,0,0,0


In [27]:
for cluster in cluster_list_renamed:
    if cluster not in similarity_df.index:
        new_row = []
        for col in similarity_df.columns:
            if cluster == col:
                new_row.append(1)
            else:
                new_row.append(0)
        similarity_df.loc[cluster] = new_row
        
similarity_df

,BGC0000001,BGC0000002,BGC0000003,BGC0000004,BGC0000006,BGC0000007,BGC0000008,BGC0000009,BGC0000011,BGC0000012,...,main-chr.region003,main-chr.region004,main-chr.region005,main-chr.region006,main-chr.region007,main-chr.region008,main-chr.region009,main-chr.region010,main-chr.region011,pls1.region001
BGC0000004,0,0,0,1.000000,0.976886,0.952463,0.975877,0.880091,0,0,...,0.0,0,0,0,0,0,0,0,0,0
BGC0000006,0,0,0,0.976886,1.000000,0.972055,0.997262,0.894729,0,0,...,0.0,0,0,0,0,0,0,0,0,0
BGC0000007,0,0,0,0.952463,0.972055,1.000000,0.971047,0.893030,0,0,...,0.0,0,0,0,0,0,0,0,0,0
BGC0000008,0,0,0,0.975877,0.997262,0.971047,1.000000,0.894099,0,0,...,0.0,0,0,0,0,0,0,0,0,0
BGC0000009,0,0,0,0.880091,0.894729,0.893030,0.894099,1.000000,0,0,...,0.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
main-chr.region008,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0.0,0,0,0,0,1,0,0,0,0
main-chr.region009,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0.0,0,0,0,0,0,1,0,0,0
main-chr.region010,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0.0,0,0,0,0,0,0,1,0,0
main-chr.region011,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0.0,0,0,0,0,0,0,0,1,0


# Filtering and merging dataframes

In [28]:
indexes_to_keep = []

for index in similarity_df.index:
    if index in class_df.index:
        indexes_to_keep.append(index)
        
print(len(indexes_to_keep))

class_df = class_df.loc[indexes_to_keep]

class_df

1740


,PKS,Other,Alkaloid,Oligosaccharide,Terpene,NRPS,Cyclitol,Aminocoumarin,Betalactam,Siderophore,Pyrrolobenzodiazepine,RiPP,Butyrolactone,Nucleoside,Phenazine,Aminoglycoside,tRNA_derived,Phosphonate,Minor,Phenolic
BGC0000004,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BGC0000006,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BGC0000007,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BGC0000008,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BGC0000009,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
main-chr.region008,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
main-chr.region009,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
main-chr.region010,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
main-chr.region011,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
similarity_df = similarity_df.loc[indexes_to_keep]

similarity_df

,BGC0000001,BGC0000002,BGC0000003,BGC0000004,BGC0000006,BGC0000007,BGC0000008,BGC0000009,BGC0000011,BGC0000012,...,main-chr.region003,main-chr.region004,main-chr.region005,main-chr.region006,main-chr.region007,main-chr.region008,main-chr.region009,main-chr.region010,main-chr.region011,pls1.region001
BGC0000004,0,0,0,1.000000,0.976886,0.952463,0.975877,0.880091,0,0,...,0.0,0,0,0,0,0,0,0,0,0
BGC0000006,0,0,0,0.976886,1.000000,0.972055,0.997262,0.894729,0,0,...,0.0,0,0,0,0,0,0,0,0,0
BGC0000007,0,0,0,0.952463,0.972055,1.000000,0.971047,0.893030,0,0,...,0.0,0,0,0,0,0,0,0,0,0
BGC0000008,0,0,0,0.975877,0.997262,0.971047,1.000000,0.894099,0,0,...,0.0,0,0,0,0,0,0,0,0,0
BGC0000009,0,0,0,0.880091,0.894729,0.893030,0.894099,1.000000,0,0,...,0.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
main-chr.region008,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0.0,0,0,0,0,1,0,0,0,0
main-chr.region009,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0.0,0,0,0,0,0,1,0,0,0
main-chr.region010,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0.0,0,0,0,0,0,0,1,0,0
main-chr.region011,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,...,0.0,0,0,0,0,0,0,0,1,0


In [30]:
merged_training_df = class_df.join(similarity_df, how='outer')

merged_training_df

,PKS,Other,Alkaloid,Oligosaccharide,Terpene,NRPS,Cyclitol,Aminocoumarin,Betalactam,Siderophore,...,main-chr.region003,main-chr.region004,main-chr.region005,main-chr.region006,main-chr.region007,main-chr.region008,main-chr.region009,main-chr.region010,main-chr.region011,pls1.region001
BGC0000004,1,0,0,0,0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0,0,0
BGC0000006,1,0,0,0,0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0,0,0
BGC0000007,1,0,0,0,0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0,0,0
BGC0000008,1,0,0,0,0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0,0,0
BGC0000009,1,0,0,0,0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
main-chr.region008,0,0,0,0,0,1,0,0,0,0,...,0.0,0,0,0,0,1,0,0,0,0
main-chr.region009,0,0,0,0,1,0,0,0,0,0,...,0.0,0,0,0,0,0,1,0,0,0
main-chr.region010,0,0,0,0,0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,1,0,0
main-chr.region011,1,0,0,0,0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0,1,0


# Creating label column

In [31]:
label_col = []

for i,r in merged_training_df.iterrows():
    if i in mibig_activity_dict:
        label_col.append(mibig_activity_dict[i])
    else:
        if 'BGC' not in i:
            label_col.append('cena_bgc')
        else:
            label_col.append('unlabeled')
        
label_col[0]

'unlabeled'

In [32]:
merged_training_df['label'] = label_col

labeled_training_df = merged_training_df[merged_training_df['label'] != 'unlabeled']

labeled_training_df

,PKS,Other,Alkaloid,Oligosaccharide,Terpene,NRPS,Cyclitol,Aminocoumarin,Betalactam,Siderophore,...,main-chr.region004,main-chr.region005,main-chr.region006,main-chr.region007,main-chr.region008,main-chr.region009,main-chr.region010,main-chr.region011,pls1.region001,label
BGC0000020,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cytotoxic
BGC0000029,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,antibacterial
BGC0000031,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,antibacterial-cytotoxic
BGC0000034,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,antifungal
BGC0000035,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,antibacterial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
main-chr.region008,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,cena_bgc
main-chr.region009,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,cena_bgc
main-chr.region010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,cena_bgc
main-chr.region011,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,cena_bgc


In [33]:
merged_training_df[merged_training_df['label'] == 'cena_bgc']

,PKS,Other,Alkaloid,Oligosaccharide,Terpene,NRPS,Cyclitol,Aminocoumarin,Betalactam,Siderophore,...,main-chr.region004,main-chr.region005,main-chr.region006,main-chr.region007,main-chr.region008,main-chr.region009,main-chr.region010,main-chr.region011,pls1.region001,label
NZ_CP012036.1.region006,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cena_bgc
NZ_CP012036.1.region008,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cena_bgc
NZ_CP023278.1.region001,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cena_bgc
NZ_CP023278.1.region010,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cena_bgc
NZ_CP023278.1.region020,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cena_bgc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
main-chr.region008,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,cena_bgc
main-chr.region009,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,cena_bgc
main-chr.region010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,cena_bgc
main-chr.region011,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,cena_bgc


In [34]:
np.unique(labeled_training_df['label'])

array(['antibacterial', 'antibacterial-antifungal',
       'antibacterial-antifungal-cytotoxic', 'antibacterial-cytotoxic',
       'antifungal', 'antifungal-cytotoxic', 'antihelmintic',
       'antioxidant', 'antiprotozoa', 'antiviral', 'cena_bgc',
       'cytotoxic', 'cytotoxic-unknown', 'herbicide', 'inhibitor',
       'other', 'pigment', 'regulatory', 'siderophore', 'unknown'],
      dtype=object)

# Predicting the unknown

In [35]:
unlabeled_training_df = merged_training_df[merged_training_df['label'] == 'cena_bgc']

unlabeled_training_df

,PKS,Other,Alkaloid,Oligosaccharide,Terpene,NRPS,Cyclitol,Aminocoumarin,Betalactam,Siderophore,...,main-chr.region004,main-chr.region005,main-chr.region006,main-chr.region007,main-chr.region008,main-chr.region009,main-chr.region010,main-chr.region011,pls1.region001,label
NZ_CP012036.1.region006,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cena_bgc
NZ_CP012036.1.region008,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cena_bgc
NZ_CP023278.1.region001,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cena_bgc
NZ_CP023278.1.region010,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cena_bgc
NZ_CP023278.1.region020,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cena_bgc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
main-chr.region008,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,cena_bgc
main-chr.region009,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,cena_bgc
main-chr.region010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,cena_bgc
main-chr.region011,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,cena_bgc


In [36]:
unknown_bgcs = unlabeled_training_df.index

In [37]:
labeled_training_df = labeled_training_df[labeled_training_df['label'] != 'cena_bgc']

In [38]:
def running_knn(training_df,testing_df,k_value):
    j_col = []
    X_div = training_df.drop("label", axis=1)
    y_div = training_df["label"]
    nbrs = NearestNeighbors(n_neighbors=k_value, algorithm='ball_tree').fit(X_div,y_div)
    distances, indices = nbrs.kneighbors(testing_df)
    y_div = y_div.reset_index(drop=True)
    neighbors_array = []
    for item in indices:
        candidate_list = []
        for i in range(k_value):
            candidate_list.append(y_div[item[i]])
        neighbors_array.append(candidate_list)
    neighbors_array = np.asarray(neighbors_array)
    return neighbors_array,indices
    

neighbors_array,indices = running_knn(labeled_training_df,unlabeled_training_df.drop('label',axis=1),1)

bgc_col,bioact_col = [],[]

for i,item in enumerate(neighbors_array):
    if i < 5:
        print(unlabeled_training_df.index[i],item)
    bgc_col.append(unlabeled_training_df.index[i])
    bioact_col.append(item)
    
print(len(neighbors_array),len(bgc_col),len(bioact_col))

NZ_CP012036.1.region006 ['antibacterial']
NZ_CP012036.1.region008 ['unknown']
NZ_CP023278.1.region001 ['unknown']
NZ_CP023278.1.region010 ['unknown']
NZ_CP023278.1.region020 ['unknown']
126 126 126


In [39]:
jaccard_col,homol_col = [],[]

def get_binary(fingerprint):
    new_row = []
    for index,item in enumerate(fingerprint):
        if item != 0:
            new_row.append(1)
        else:
            new_row.append(0)
    fingerprint = new_row
    return fingerprint

for i,bgc_id in enumerate(unlabeled_training_df.index):
    count = 0
    train_index = int(indices[i][0])
    if i < 5:
        print(i,bgc_id,train_index)
    for j,r in labeled_training_df.iterrows():
        if count == train_index:
            training_fp = labeled_training_df.loc[j].drop('label')
            homol_col.append(j)
        count += 1
    testing_fp = unlabeled_training_df.drop('label',axis=1).loc[bgc_id]
    training_binary = get_binary(training_fp)
    testing_binary = get_binary(testing_fp)
    jaccard_index = jaccard_score(training_binary,testing_binary)
    jaccard_col.append(jaccard_index)
    print(jaccard_index)
    
print(len(jaccard_col))

0 NZ_CP012036.1.region006 426
0.25
1 NZ_CP012036.1.region008 173
1.0
2 NZ_CP023278.1.region001 272
0.8
3 NZ_CP023278.1.region010 459
0.25
4 NZ_CP023278.1.region020 72
0.75
0.25
0.25
0.2
0.4
0.25
0.7777777777777778
0.25
0.7142857142857143
0.3333333333333333
0.4
0.25
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.5
0.3333333333333333
0.3333333333333333
0.5
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.5
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.25
0.5
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.4
0.5
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.

In [40]:
frames = {'bgc':bgc_col,'bioactivity':bioact_col,'jaccard_score':jaccard_col,'homolog':homol_col}

valid_jaccard_df = pd.DataFrame(frames)

valid_jaccard_df = valid_jaccard_df[valid_jaccard_df['jaccard_score'] >= 0.7]

valid_jaccard_df = valid_jaccard_df.sort_values(by='jaccard_score',ascending=False)

valid_jaccard_df = valid_jaccard_df.reset_index(drop=True)

valid_jaccard_df

,bgc,bioactivity,jaccard_score,homolog
0,NZ_CP012036.1.region008,[unknown],1.000000,BGC0001028
1,NZ_CP023278.1.region001,[unknown],0.800000,BGC0001705
2,c00035_NODE_35...region001,[antibacterial-antifungal-cytotoxic],0.777778,BGC0001612
3,NZ_CP023278.1.region020,[unknown],0.750000,BGC0000427
4,chrpls1.region003,[cytotoxic],0.714286,BGC0001016


In [41]:
for key in bigscape_dict:
    if 'c00035_NODE_35...region001' in str(bigscape_dict[key]):
        print(bigscape_dict[key])

['BGC0001594.1.region001', 'BGC0000668.1.region001', 'BGC0001595.1.region001', 'c00035_NODE_35...region001', 'BGC0001501.1.region001', 'BGC0001612.1.region001', 'BGC0001126.1.region001']


In [42]:
for i,item in unlabeled_training_df.drop('label',axis=1).loc['c00035_NODE_35...region001'].iteritems():
    if item > 0:
        print(i,item)

Other 1
BGC0000668 0.846348911523819
BGC0001126 0.9291904419660569
BGC0001501 0.92709369212389
BGC0001594 0.7142514586448669
BGC0001595 0.7423691153526306
BGC0001612 0.9291904419660569
c00035_NODE_35...region001 1.0


In [43]:
for i,item in labeled_training_df.drop('label',axis=1).loc['BGC0001612'].iteritems():
    if item > 0:
        print(i,item)

Alkaloid 1
BGC0000668 0.8584925383329391
BGC0001126 1.0
BGC0001501 0.9679582305252552
BGC0001594 0.9150469452142715
BGC0001595 0.8034721612930298
BGC0001612 1.0
c00035_NODE_35...region001 0.9291904419660569


# BiG-SCAPE dereplication

In [44]:
for key in bigscape_dict:
    for item in bigscape_dict[key]:
        if 'BGC' not in item:
            print(key,bigscape_dict[key])

GCF52 ['BGC0000427.1.region001', 'NZ_CP023278.1.region020']
GCF81 ['BGC0001594.1.region001', 'BGC0000668.1.region001', 'BGC0001595.1.region001', 'c00035_NODE_35...region001', 'BGC0001501.1.region001', 'BGC0001612.1.region001', 'BGC0001126.1.region001']
GCF92 ['NZ_CP012036.1.region006', 'NZ_CP023278.1.region022', 'NZ_MTPU01000053.region001', 'BGC0000869.1.region001']
GCF92 ['NZ_CP012036.1.region006', 'NZ_CP023278.1.region022', 'NZ_MTPU01000053.region001', 'BGC0000869.1.region001']
GCF92 ['NZ_CP012036.1.region006', 'NZ_CP023278.1.region022', 'NZ_MTPU01000053.region001', 'BGC0000869.1.region001']
GCF117 ['chrpls1.region003', 'BGC0001667.1.region001', 'NZ_CP023278.1.region001', 'BGC0001016.1.region001', 'BGC0001705.1.region001']
GCF117 ['chrpls1.region003', 'BGC0001667.1.region001', 'NZ_CP023278.1.region001', 'BGC0001016.1.region001', 'BGC0001705.1.region001']
GCF121 ['BGC0001028.1.region001', 'NZ_CP012036.1.region008']
GCF177 ['BGC0001748.1.region001', 'NZ_CP023278.1.region010']
GCF188 ['

In [45]:
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
run_time = "{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)
print(run_time)

00:03:25.28
